This Notebook connects to a mongodb database, connects to the Lakeshore temperature monitors that we are using, creates a data object by scanning these instruments, and inserts them into the MongoDB database.

In [1]:
import time
import numpy as np
from datetime import datetime
from calendar import timegm
from pymongo import MongoClient, ASCENDING, DESCENDING
import pyvisa
import serial

# Setup and configure Mongo

In [2]:
def dbConnect():
    client = MongoClient('mongodb://localhost/')
    db = client.data
    collection = db.adr2datas
    db.collection_names()
    infocollection = db.adr2infos
    return collection, infocollection
collection,infocollection = dbConnect()

In [3]:
collection.name

u'adr2datas'

In [4]:
infocollection.full_name

u'data.adr2infos'

# Add Single Data file to DB

### Returns dict with random data

In [17]:
def rand_data():
    post = {
        "timeStamp": timegm(datetime.now().timetuple()),
        "baseTemp": .05*np.random.rand()+.05,
        "threeKTemp": 0.7*np.random.rand()+2.8,
        "sixtyKTemp": np.random.rand()+60,
        "magnetVoltage": np.random.rand(),
        "psVoltage": np.random.rand(),
        "psCurrent": np.random.rand(),
        "currentJob": "None",
        "percentComplete": 0,
        "switchState": "Closed"
    }
    return post
rand_data()

{'baseTemp': 0.087650353245358,
 'currentJob': 'None',
 'magnetVoltage': 0.4452160265629367,
 'percentComplete': 0,
 'psCurrent': 0.9102815720770526,
 'psVoltage': 0.2778003531985641,
 'sixtyKTemp': 60.429288673929356,
 'switchState': 'Closed',
 'threeKTemp': 3.2061069052552913,
 'timeStamp': 1420547121}

In [4]:
for data in collection.find().limit(10):
    print(data)

{u'_id': ObjectId('5579f0e6c860b137b03e2e39'), u'threeKTemp': 288.449, u'timeStamp': 1434040486, u'oneKTemp': 251.654, u'currentJob': u'None', u'switchState': u'Closed', u'psVoltage': 0, u'percentComplete': 0, u'psCurrent': 0, u'sixtyKTemp': 258.546, u'magnetVoltage': -0.00053, u'baseTemp': 0.0}
{u'_id': ObjectId('5579f0ecc860b137b03e2e3a'), u'threeKTemp': 288.421, u'timeStamp': 1434040492, u'oneKTemp': 251.644, u'currentJob': u'None', u'switchState': u'Closed', u'psVoltage': 0, u'percentComplete': 0, u'psCurrent': 0, u'sixtyKTemp': 258.431, u'magnetVoltage': -0.00053, u'baseTemp': 0.0}
{u'_id': ObjectId('5579f0f1c860b137b03e2e3b'), u'threeKTemp': 288.397, u'timeStamp': 1434040497, u'oneKTemp': 251.945, u'currentJob': u'None', u'switchState': u'Closed', u'psVoltage': 0, u'percentComplete': 0, u'psCurrent': 0, u'sixtyKTemp': 258.328, u'magnetVoltage': -0.00052, u'baseTemp': 0.0}
{u'_id': ObjectId('5579f0f6c860b137b03e2e3c'), u'threeKTemp': 288.372, u'timeStamp': 1434040502, u'oneKTemp':

## Get Data from Instruments

### Create Instrument Objects

In [3]:
def connectInstruments():
    rm = pyvisa.ResourceManager()
    rm.list_resources()
    inst1 = rm.open_resource('GPIB0::1::INSTR', read_termination = '\x00')
    inst2 = rm.open_resource('GPIB0::2::INSTR', read_termination = '\x00')
    inst3 = rm.open_resource('GPIB0::6::INSTR', read_termination = '\x00')
    return [inst1,inst2,inst3]

In [22]:
x = connectInstruments()
x[0].ask('*IDN?')

u'LSCI,MODEL340,341911,042304'

In [23]:
inst[0].encoding 

u'ascii'

### Get Data Object

In [17]:
# inst = connectInstruments()
# for i in inst:
#     i.encoding = 'utf-8'
#     print(i.encoding)
    
def get_data(inst):
    post = {
        "timeStamp": timegm(datetime.now().timetuple()),
        "baseTemp": float(inst[1].ask('KRDG? B')),
        "oneKTemp": float(inst[0].ask('KRDG? C')),
        "threeKTemp": float(inst[0].ask('KRDG? A')),
        "sixtyKTemp": float(inst[0].ask('KRDG? B')),
        "magnetVoltage": float(inst[0].ask('SRDG? D')),
        "psVoltage": 0,
        "psCurrent": 0,
        "currentJob": "None",
        "percentComplete": 0,
        "switchState": "Closed"
    }
    return post
get_data(connectInstruments())

{'baseTemp': 0.0,
 'currentJob': 'None',
 'magnetVoltage': -0.00013,
 'oneKTemp': 3.4709,
 'percentComplete': 0,
 'psCurrent': 0,
 'psVoltage': 0,
 'sixtyKTemp': 52.002,
 'switchState': 'Closed',
 'threeKTemp': 3.5476,
 'timeStamp': 1435584042}

In [19]:
float(inst[1].ask('KRDG? B'))*1000

0.0

In [67]:
'abb0b7b8aeb1b3b6c5abb0'.decode("hex")

'\xab\xb0\xb7\xb8\xae\xb1\xb3\xb6\xc5\xab\xb0'

###Validate Data, Determine Fridge Status

In [ ]:
def fridgeStatus(data, infocollection):
    if (data['sixtyKTemp'] >= 289.9):
        infocollection.

### Update every 5 seconds

In [ ]:
inst = connectInstruments()
while True:
    try:
        temp = get_data(inst)
        
    except UnicodeDecodeError:
        print("Read error at " + datetime.now().strftime('%H:%M:%S'))
        inst = connectInstruments()
        
    else:
        if (temp['sixtyKTemp'] <= 300):
            try:
                collection.insert(temp)
            except:
                print("Database error at " + datetime.now().strftime('%H:%M:%S'))
                
    time.sleep(5)

In [ ]:
while True:
    temp = get_data()
    if (temp['sixtyKTemp'] <= 290):
            collection.insert(temp)
    time.sleep(5)

In [10]:
temp['sixtyKTemp']

296.278

##Log File

In [ ]:
logFileName = 'log.txt'

##Scrape Data + Magnet Control

In [ ]:
while True:
    try:
        temp = get_data(inst)
        
    except UnicodeDecodeError:
        print("Read error at " + datetime.now().strftime('%H:%M:%S'))
        inst = connectInstruments()
        
    else:
        if (temp['sixtyKTemp'] <= 290):
            try:
                collection.insert(temp)
            except:
                print("Database error at " + datetime.now().strftime('%H:%M:%S'))
    
                
    time.sleep(5)